In [ ]:
# Importing Library

import numpy as np

## Initial Condiitons

def initialise():
    global pc, cache, mem, mem_temp, direct_hash, direct_map
    
    cache = [np.array([int, np.empty(4), 0], dtype=object), 
         np.array([int, np.empty(4), 0], dtype=object), 
         np.array([int, np.empty(4), 0], dtype=object), 
         np.array([int, np.empty(4), 0], dtype=object)] 
    
    mem = [[0]*4 for x in range(32)]
    mem_temp = [[0]*4 for x in range(32)]
    
    direct_hash = [[] for x in range(4)]
    direct_map = {}
    for i in range(32):
        direct_map[i] = i % 4

    for i in direct_map:
        direct_hash[direct_map[i]].append(i)
     
    pc =  '00' 

# Base conversions

def toHex(n: int) -> str: 
        digits = "0123456789ABCDEF"
        x = (n % 16)
        rest = n // 16

        if (rest == 0):
            ans = digits[x]
        else:
            ans = (toHex(rest) + digits[x])

        return ans

## Binary to decimal

def bin_to_dec(n: int) -> int:
    return (int(str(n), 2))

## Decimal to Binary

def dec_to_bin(n: int) -> int:
    return (int("{0:b}".format(n)))

## Hex to Decimal

def hex_to_dec(n: str) -> int:
    return int(n, 16)

## Decimal to Hex

def dec_to_hex(n: int) -> str:
    temp = n
        
    ans = toHex(temp)
    
    if n < 16:
        ans = ans.zfill(2)
    
    return ans

hex_to_dec('0')

# Basic address partitioning and cache access

## Partitioning the address byte

def partition(i):
    b_offset = int("{:02d}".format(i%100))
    b_no = int(i / 100)
    tag = int(b_no / 100)
    line_no = int("{:02d}".format(b_no%100))
    
    b_no = bin_to_dec(b_no)
    b_offset = bin_to_dec(b_offset)
    line_no = bin_to_dec(line_no)
    tag = bin_to_dec(tag)
    
    return tag, line_no, b_no, b_offset

## Check cache return data

def check_cache2(tag, line_no, b_offset):
    valid = 0
    
    required_bno = direct_hash[line_no][tag]
    
    if(cache[line_no][0] == required_bno): #hit
        valid = 1
        return (cache[line_no][1][b_offset])
    else: #Miss
        valid = 0
        cache[line_no][0] = required_bno
        cache[line_no][1] = mem[required_bno]
        return (mem[required_bno][b_offset])

# Full read and write functinos

## Full read from hex address

def read(addr: str, offset: int) -> str:
    addr_dec = hex_to_dec(addr) + offset
    addr_bin = dec_to_bin(addr_dec)
    tag, line_number, block_number, block_offset = partition(addr_bin)
    data = check_cache2(tag, line_number, block_offset)
#     if(type(data) == str):
    data = str(data)
    data = data.upper()
    return data

## Full write to hex address

def write(addr, offset: int, data: str) -> None:
    global mem
    
    if(type(addr) == str):
        addr = hex_to_dec(addr) 
    addr += offset
    addr_bin = dec_to_bin(addr)
    tag, line_number, block_number, block_offset = partition(addr_bin)
    mem[block_number][block_offset] = data

# Pipeline stages

## Decode function

def decode():
    global pc
    
    op1 = read(pc, 1)
    op2 = read(pc, 2)
    dest = read(pc, 3)
    op1 = hex_to_dec(op1)
    op2 = hex_to_dec(op2)
    dest_dec = hex_to_dec(dest)
    
    return op1, op2, dest_dec

## Execute

def execute(IR: str, op1: int, op2: int) -> str:
    
    if(IR == '87'): # add
        res = dec_to_hex(op1 + op2)

    elif(IR == '97'): # sbb
        res = dec_to_hex(op1 - op2)

    elif(IR == 'A7'): # bitwise and
        res = dec_to_hex(op1 & op2)

    elif(IR == 'B7'): # bitwise or
        res = dec_to_hex(op1 | op2)

    elif(IR == 'AF'): # bitwise xor
        res = dec_to_hex(op1 ^ op2)
        
    else:
        res = 'ee'
    
    return res

## Hex increment

def inc(data: str) -> str:
    data_dec = hex_to_dec(data)
    result_dec = data_dec + 1
    result_hex = dec_to_hex(result_dec)
    
    return result_hex

# Main States

## Program(enter opcode and data)

def program():
    global start_og, mem, mem_temp
    
    print("Enter starting address in hex (up to 7F): ")
    start_og = input()
    start = hex_to_dec(start_og)


    print("Enter q at any stage to quit without saving program and 'wq' to save and quit. Otherwise enter Opcode or data.")

    while(1):

        start_bin = dec_to_bin(start)
        tag, line_number, block_number, block_offset = partition(start_bin)

        data = input(f"code {dec_to_hex(start)}>")

        if(data == 'q'):
            break
        elif(data == 'wq'):
            mem = mem_temp
            break
        else:
            mem_temp[block_number][block_offset] = data
            start += 1

## Execute(run the code)

def run_code():
    global pc, start_og
    pc = start_og
    IR = '00'

    while(1):
        # Fetch operation
        IR = read(pc, 0)

        if(IR == '76'): 
            print("execution complete")
            break

        else:

            # Decode
            op1, op2, dest = decode()

            # Execute
            res = execute(IR, op1, op2)

            # Write back    
            write(dest, 0, res)

            # Increment pc to next instruction
            for i in range(4):
                pc = inc(pc)

## Debug

def debug():
    while(1):
        debug_temp = input("debug>")
        
        if(debug_temp == 'cache'):
            for line in cache:
                print(*line)
        
        elif(debug_temp == 'memory'):
            for line in mem:
                print(*line)
                
        elif(debug_temp == 'pc'):
            print(pc)
                
        elif(debug_temp == 'q'):
            break

# Manual

def manual():
    print(f"{'-'*20}HELP{'-'*20}")
    print("User mode: General mode. Enter 'q' in any other mode to get back to user mode.")
    print("Code mode: Mode to write a program into memory. Enter 'code' in user mode to enter code mode.")
    print("Execute mode: Mode to run the program from memory. Enter 'execute' in user mode to enter execute mode.")
    print("Reset: Enter 'reset' in user mode to reset memory cache and program counter.")
    print("Debug mode: Mode to check program resutls. Enter 'debug' in user mode to enter code mode. In this mode, enter 'cache' to view cache, enter 'memory' to view memory and enter 'pc' to view program counter")
    print("Enter 'q!' in User mode to quit the program.")

# Main

def main():
    while(1):
        temp = input("user>")
        
        if(temp == 'code'):
            program()
            
        elif(temp == 'execute'):
            run_code()
        
        elif(temp == 'debug'):
            debug()
            
        elif(temp == 'reset'):
            initialise()
            
        elif(temp == 'q!'):
            break
            
        else:
            manual()

main()

user> help


--------------------HELP--------------------
User mode: General mode. Enter 'q' in any other mode to get back to user mode.
Code mode: Mode to write a program into memory. Enter 'code' in user mode to enter code mode.
Execute mode: Mode to run the program from memory. Enter 'execute' in user mode to enter execute mode.
Reset: Enter 'reset' in user mode to reset memory cache and program counter.
Debug mode: Mode to check program resutls. Enter 'debug' in user mode to enter code mode. In this mode, enter 'cache' to view cache, enter 'memory' to view memory and enter 'pc' to view program counter
Enter 'q!' in User mode to quit the program.


user> code


Enter starting address in hex (up to 7F): 


 00


Enter q at any stage to quit without saving program and 'wq' to save and quit. Otherwise enter Opcode or data.


code 00> 87
code 01> 0b
code 02> 0a
code 03> 7f
code 04> 97
code 05> 0b
code 06> 0a
code 07> 7e
code 08> a7
code 09> 0b
code 0A> 0a
code 0B> 7d
code 0C> b7
code 0D> 0b
code 0E> 0a
code 0F> 7c
code 10> af
code 11> 0b
code 12> 0a
code 13> 7b
code 14> 76
code 15> wq
user> execute


execution complete


user> debug
debug> memory


87 0b 0a 7f
97 0b 0a 7e
a7 0b 0a 7d
b7 0b 0a 7c
af 0b 0a 7b
76 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 01
0B 0A 01 15
